# Day 3 - Conversational AI - aka Chatbot!

In [8]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import google.generativeai
import ollama

In [ ]:
#constants
MODEL="llama3.2"

In [ ]:
def stream_ollama(prompt, model_name=MODEL):
    try:
        response = ollama.chat(model=MODEL, messages=[{'role': 'user', 'content': prompt}], stream=True)
        result = ""
        for chunk in response:
            # Check if 'message' key exists and 'content' key exists within 'message'
            if 'message' in chunk and 'content' in chunk['message']:
                result += chunk['message']['content']
                yield result
            # Optionally handle chunks that don't have the expected structure
            # else:
            #     print(f"Skipping chunk with unexpected structure: {chunk}")
    except Exception as e:
        yield f"Error generating response from Ollama ({model_name}): {e}"

In [9]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key not set
Anthropic API Key not set
Google API Key exists and begins AIzaSyCn


In [10]:
# This is the set up code for Gemini

google.generativeai.configure()

In [11]:
system_message = "You are a helpful assistant"

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [29]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

# Student Octavio O. has pointed out that this isn't quite as straightforward for Claude -
# see the excellent contribution in community-contributions "Gradio_issue_with_Claude" that handles Claude.
import google.generativeai as genai
# Define the system message outside the function if it's constant
system_message = "You are a helpful AI assistant."

def chat(message, history):
  # Construct the messages list, ensuring history elements are also correctly formatted
  # The error indicates the structure of history items is not as expected by the API
  # We need to convert history from Gradio's format (usually list of lists [user_msg, bot_msg])
  # to the format expected by Gemini API: list of dicts [{"role": "...", "parts": [...]}]

  formatted_history = []
  for turn in history:
    # Gradio history can sometimes have odd structures, ensure we handle single elements too
    if isinstance(turn, list) and len(turn) == 2:
        human, ai = turn
        formatted_history.append({"role": "user", "parts": [human]})
        formatted_history.append({"role": "model", "parts": [ai]}) # Gemini uses 'model' for AI
    elif isinstance(turn, str): # Handle cases where a turn might just be a string (less common for history but good practice)
         formatted_history.append({"role": "user", "parts": [turn]})
    # Add other handling for different history structures if needed

  messages = [{"role": "user", "parts": [system_message]}] + formatted_history + [{"role": "user", "parts": [message]}]

  print("History is:")
  print(history)
  print("And messages is:")
  print(messages)

  model = genai.GenerativeModel('gemini-1.5-flash-latest')
  stream = model.generate_content(messages, stream=True)
  response = ""
  for chunk in stream:
    response += chunk.text
    yield response


## And then enter Gradio's magic!

In [30]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'user', 'parts': ['You are a helpful AI assistant.']}, {'role': 'user', 'parts': ['f']}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'f', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "I'm ready for your request.  What can I help you with?", 'options': None}]
And messages is:
[{'role': 'user', 'parts': ['You are a helpful AI assistant.']}, {'role': 'user', 'parts': ['dd']}]
History is:
[{'role': 'user', 'metadata': None, 'content': 'f', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "I'm ready for your request.  What can I help you with?", 'options': None}, {'role': 'user', 'metadata': None, 'content': 'dd', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': '"dd" is a command-line utility in Unix-like operating systems.  It\'s used for copying and converting files.  Because it\'s powerful and can easily overwrite data, it\'s important to use it carefully.  Without mo